In [1]:
import os
import sys
import torch
import numpy as np

import datetime
import logging
import importlib
import shutil
import argparse

from pathlib import Path
# from tqdm import tqdm
from data_utils.MoleculeDataSet import PointCloudMoleculeDataSet, load_and_align_QM7
from data_utils.plotting_utils import make_training_progress_plot, make_predictions_plot

# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = '/Users/owen/projects/Pointnet_Pointnet2_pytorch/'
sys.path.append(os.path.join(ROOT_DIR, 'models'))


from train_PointNet_QM7 import test

# Set up paths

I have pre-trained a model for 10 epochs and saved the model's weights at each epoch in `CHECKPOINT_DIR`.

In [3]:
module = importlib.import_module('pointnet2_reg_msg')

In [4]:
CHECKPOINT_DIR = '/local/meliao/projects/Pointnet_Pointnet2_pytorch/log/tmp/regression_QM7/2022-12-02_09-26/checkpoints'

In [16]:
def compare_two_models(checkpoint_1_fp: str, checkpoint_2_fp: str) -> float:
    model_1 = module.PointNet2MSGModel(n_centroids_1=10,
                                        msg_radii_1=[2., 4., 8.],
                                        msg_nsample_1=[4, 8, 16],
                                        n_centroids_2=4,
                                        msg_radii_2=[2., 4., 8.],
                                        msg_nsample_2=[2, 4, 8],
                                        in_channels=5,
                                        out_channels=1)
    checkpoint_1_data = torch.load(checkpoint_1_fp)
    model_1.load_state_dict(checkpoint_1_data['model_state_dict'])
    
    model_2 = module.PointNet2MSGModel(n_centroids_1=10,
                                        msg_radii_1=[2., 4., 8.],
                                        msg_nsample_1=[4, 8, 16],
                                        n_centroids_2=4,
                                        msg_radii_2=[2., 4., 8.],
                                        msg_nsample_2=[2, 4, 8],
                                        in_channels=5,
                                        out_channels=1)
    
    checkpoint_2_data = torch.load(checkpoint_2_fp)
    model_2.load_state_dict(checkpoint_2_data['model_state_dict'])
    
    
    param_lst_1 = [x for x in model_1.parameters()]
    param_lst_2 = [x for x in model_2.parameters()]
    
    lst_of_l2_diffs = []
    lst_of_sizes = []
    for weight_1, weight_2 in zip(param_lst_1, param_lst_2):
        diff = torch.linalg.norm(weight_1 - weight_2)
        size = weight_1.size()
        print(f"DIFF: {diff}, SIZE: {size}, and NAME: {weight_1.name}")
        lst_of_l2_diffs.append(diff)
        lst_of_sizes.append(size)
    return None

In [17]:
fp_1 = os.path.join(CHECKPOINT_DIR, 'epoch_0.pth')
fp_2 = os.path.join(CHECKPOINT_DIR, 'epoch_1.pth')

out = compare_two_models(fp_1, fp_2)

DIFF: 0.09134529531002045, SIZE: torch.Size([32, 8, 1, 1]), and NAME: None
DIFF: 0.03337373957037926, SIZE: torch.Size([32]), and NAME: None
DIFF: 0.18379178643226624, SIZE: torch.Size([32, 32, 1, 1]), and NAME: None
DIFF: 0.02981395646929741, SIZE: torch.Size([32]), and NAME: None
DIFF: 0.24057872593402863, SIZE: torch.Size([64, 32, 1, 1]), and NAME: None
DIFF: 0.03959996625781059, SIZE: torch.Size([64]), and NAME: None
DIFF: 0.11221829056739807, SIZE: torch.Size([64, 8, 1, 1]), and NAME: None
DIFF: 0.04000828415155411, SIZE: torch.Size([64]), and NAME: None
DIFF: 0.33089372515678406, SIZE: torch.Size([64, 64, 1, 1]), and NAME: None
DIFF: 0.04048740491271019, SIZE: torch.Size([64]), and NAME: None
DIFF: 0.46834635734558105, SIZE: torch.Size([128, 64, 1, 1]), and NAME: None
DIFF: 0.06072547286748886, SIZE: torch.Size([128]), and NAME: None
DIFF: 0.12170299142599106, SIZE: torch.Size([64, 8, 1, 1]), and NAME: None
DIFF: 0.037835754454135895, SIZE: torch.Size([64]), and NAME: None
DIFF: 

# Data Loading

In [3]:
DATA_FP = '/local/meliao/projects/invariant-random-features/data/qm7/qm7.mat'
N_TRAIN = 500
N_TEST = 100
BATCH_SIZE = 128

train_dset, val_dset, test_dset = load_and_align_QM7(fp=DATA_FP,
                                                        n_train=N_TRAIN,
                                                        n_test=N_TEST,
                                                        validation_set_fraction=0.1)

In [4]:
train_loader = torch.utils.data.DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dset, batch_size=BATCH_SIZE, shuffle=True)

# Compile model and optimizer

In [8]:
model = module.PointNet2MSGModel(n_centroids_1=10,
                                        msg_radii_1=[2., 4., 8.],
                                        msg_nsample_1=[4, 8, 16],
                                        n_centroids_2=4,
                                        msg_radii_2=[2., 4., 8.],
                                        msg_nsample_2=[2, 4, 8],
                                        in_channels=5,
                                        out_channels=1)

loss_fn = module.get_loss()


'''MOVE EVERYTHING TO THE CORRECT DEVICE'''


NameError: name 'device' is not defined